In [1]:
import folium
import random
import requests
import polyline
from scipy.spatial import distance_matrix
import numpy as np

# Division 32 - Neighborhood coordinates
ne_point = (43.803239, -79.396278)
se_point = (43.727556, -79.382024)
sw_point = (43.710802, -79.469791)
nw_point = (43.783980, -79.484467)

# Generate random points
points = []
for _ in range(5):
    lat = random.uniform(sw_point[0], ne_point[0])
    lon = random.uniform(sw_point[1], ne_point[1])
    points.append((lat, lon))

# Create map centered on the neighborhood
map_center = [(ne_point[0] + sw_point[0]) / 2, (ne_point[1] + sw_point[1]) / 2]
map_toronto = folium.Map(location=map_center, zoom_start=12, tiles="cartodb dark_matter") #bright tile - positron 

# Add markers for the random points
for point in points:
    folium.Marker(location=point).add_to(map_toronto)

# Read API key from file
with open("api_key.txt", "r") as file:
    api_key = file.read().strip()

# Get distance matrix between points
dist_matrix = distance_matrix(points, points)

# Nearest Neighbor algorithm for TSP
num_points = len(points)
visited = np.zeros(num_points, dtype=bool)
route = []
current_point = 0
for _ in range(num_points):
    route.append(points[current_point])
    visited[current_point] = True
    next_point = None
    min_distance = float('inf')
    for i in range(num_points):
        if not visited[i] and dist_matrix[current_point, i] < min_distance:
            min_distance = dist_matrix[current_point, i]
            next_point = i
    current_point = next_point

# Get driving directions between points
directions = []
for i in range(len(route) - 1):
    start = route[i]
    end = route[i + 1]
    url = f"https://maps.googleapis.com/maps/api/directions/json?origin={start[0]},{start[1]}&destination={end[0]},{end[1]}&key={api_key}"
    response = requests.get(url)
    data = response.json()
    route_data = data["routes"][0]["overview_polyline"]["points"]
    directions.extend(polyline.decode(route_data))

# Connect the points with a route
folium.PolyLine(locations=directions, color='blue', weight=2).add_to(map_toronto)

# Display the map
map_toronto